## 목차
- **1강 : 실습 준비**
 - EDA 및 전처리
- **2강 : 추천시스템 개요**
 - 추천시스템 목적 및 활용 사례
 - 추천 알고리즘 Overview
 - 알고리즘별 주요 특성
 - 추천 성능 지표의 이해
- **3강 : 베이스라인 추천 알고리즘 구축**
 - (Memory-based) Collaborative Filtering
 - (Model-based) Matrix Factorazation (SVD, NMF, SVDpp)
- **4강 : 추천 알고리즘 성능 고도화**
 - (Model-based) DeepLearning (DNN, NCF)
 - (Model-based) Machine Learning-based Model
- **5강 : Cold Start Issues**
 - Content-based filtering
 - Populairity-based
- **6강 : 최종 추천모델 클래스 구성**
 - Matrix Factorization(SVD), Machine Learning(GBR), Content+Poupularity 추천을 위한 클래스

# 1강 데이터 소개 및 실습 준비
- 1강에서는 모델 개발에 필요한 환경을 세팅하고 필요한 데이터셋을 준비.
- 프로젝트에 활용할 데이터에 대해 EDA 및 시각화를 통해 이해.
## 개발 환경
- **Kaggle Notebook**
    - 캐글노트북 환경에서 제공하는 라이브러리의 기준에 맞춰 코드 작성.



## 데이터셋 소개
- 소스데이터: Book Recommendation Dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data)를 한번 더 가공한 [실습 데이터: books-data](https://www.kaggle.com/datasets/liamsong/books-data)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
sns.set(style="whitegrid")
from IPython.display import Image, display

data_path = '/kaggle/input/books-data/' # 각자 경로 확인 필요
file_name = 'books.csv'

In [ ]:
df = pd.read_csv(data_path+file_name)
df.tail()

In [ ]:
df.info()

In [ ]:
df.groupby("Category").size().sort_values(ascending=False)[:10]

In [ ]:
# Preprocessing
df.columns = df.columns.str.lower() # 컬럼명 소문자 처리
df['rating'] = df['rating'].astype(float)
df['age_range'] = (df['age']/10).astype(int)*10 # 십의 자리수로 연령대 변환
df['category'] = df['category'].replace("9", np.nan) # null 로 변환
df['category'] = df['category'].str.replace("['", "").str.replace("']", "") # 불필요한 문제 제거
df['summary'] = df['summary'].replace("9", np.nan) # # null 로 변환

In [ ]:
df.tail()

## EDA (Exploratory Data Analysis)


In [ ]:
print(df.shape)
print(df.user_id.nunique()) # the number of unique users
print(df.isbn.nunique()) # the number of unique items

In [ ]:
# ISBN 별로 groupby 이후에 평점 평균, 리뷰 갯수 등 통계 만들기
# 이 데이터프레임을 추후에 여러 목적으로 활용 (추천결과 확인, Popularity-based 추천 등)
book_info = df[['isbn', 'book_title', 'year_of_publication', 'book_author', 'category']].drop_duplicates()
book_info_rating = df.groupby(['isbn']).agg(rating_mean=('rating', 'mean'), rating_count=('rating', 'size')).reset_index()

book_info = book_info.merge(book_info_rating, on='isbn').sort_values("rating_count", ascending=False)
book_info.head(10)

In [ ]:
print(book_info.shape)
print(book_info.isbn.nunique())
print(book_info.book_author.nunique())
print(book_info.category.nunique())

In [ ]:
book_info.isnull().sum()

In [ ]:
# 1 row and 2 columns suplot 생성
fig, axes = plt.subplots(1, 2, figsize=(9, 4))

# 유저의 연령대별 빈도수 분포
df[['user_id', 'age_range']].drop_duplicates()['age_range'].value_counts().sort_index().plot(kind='bar', rot=0, ax=axes[0])
axes[0].set_title('Age Range Distribution')

# 유저가 남긴 평점들의 분포
df['rating'].value_counts().sort_index().plot(kind='bar', rot=0, ax=axes[1])
axes[1].set_title('Rating Distribution')

plt.tight_layout()
plt.show()

In [ ]:
df['rating'].describe()

In [ ]:
df.head()

In [ ]:
# 분석 단위: 68092명으로 변환
df.groupby("user_id")['rating'].mean().describe()

In [ ]:
# 유저당 남긴 리뷰수의 분포
df.groupby("user_id")['rating'].size().value_counts().sort_index()[:50].plot(kind='bar', figsize=(12, 4), rot=0)

plt.title("The Count of Users' rates")
plt.tight_layout()

In [ ]:
# 유저당 평점의 평균
df.groupby("user_id")['rating'].mean().hist(bins=10, figsize=(5,4), color='green', alpha=.4)

plt.title("The Histogram of Users' rating")
plt.axvline(x=7.487156, color='b', linestyle='--')
plt.grid()
plt.tight_layout()

In [ ]:
# Top 10 Books and Authors
top_books = df['book_title'].value_counts().head(15)
top_authors = df['book_author'].value_counts().head(10)
top_cate = df['category'].value_counts().head(10)

In [ ]:
top_books

In [ ]:
top_authors

In [ ]:
top_cate

# 2강 추천시스템 개요


## 추천 알고리즘 Overview
- **Collatorative Filtering (Memory-based, Model-based), Content-based** 영역을 중심으로 실습할 예정.

## 주요 라이브러리
- 전처리 및 추출, 데이터 분석 및 간단한 모델링은 **Pandas, Numpy, Matplotlib, Seaborn**을 활용.
- [Surprise](https://surprise.readthedocs.io/en/stable/index.html) 라이브러리를 이용해 Collatorative Filtering (Memory-based) 관련 모델을 구축.
    - 해당 라이브러리는 User-based, Item-based CF 뿐만 아니라 행렬분해 알고리즘인 SVD를 이용한 Matrix Factorizaiton 모델 구축을 지원.
- Collatorative Filtering (Model-based)의 경우 [Scikit-learn](https://scikit-learn.org/stable/)를 활용.
    - Content-based Filtering도 Scikit-learn에서 제공하는 모듈을 이용.
    - Scikit-learn은 Regression, Classification 및 비지도학습 알고리즘과 피처 엔지니어링 관련 모듈을 제공.
    - Scikit-learn에서 제공하는 Regression 관련 모듈을 이용해 유저의 평점을 예측하는 모델을 구축.
- 딥러닝 기반 추천모델을 개발하기 위해 Keras 라이브러리를 이용할 예정.
    - DNN, Neural Collaborative Filtering 등을 위 라이브러리를 이용하여 실습할 예정.

    

# 3강 베이스라인 추천모델 구축

## Collaborative Filtering  
- [KNN Basic](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic) 알고리즘은 유사한 사용자들의 평가를 바탕으로, 각 사용자와의 유사도를 고려하여 가중 평균을 계산함으로써 새로운 평점을 예측.
- 즉 유사한 이웃을 찾아서 해당 이웃이 기존에 부여한 평점의 평균을 구하는데, 유사도를 가중치를 이용하여 평균을 구함.
- 예를 들어, "A"라는 사용자에게 "Life of Pi" 책의 예상 평점을 추천하려고 할때 "A"와 가장 유사한 3명의 이웃(k=3)을 찾았다고 가정.
   - B **(유사도: 0.8) -> 해당 책의 평점이 4.5**
   - C **(유사도: 0.6) -> 해당 책의 평점이 5.0**
   - D **(유사도: 0.5) -> 해당 책의 평점이 3.5**
- 그렇다면 A가 "Life of Pi"에 줄 예측 평점을 구하는 과정은
    1. 각 이웃의 평점에 유사도를 곱함.
       - **B: 4.5 * 0.8 = 3.6**
       - **C: 5.0 * 0.6 = 3.0**
       - **D: 3.5 * 0.5 = 1.75**
    2. 이 값들을 모두 더합니다: **3.6 + 3.0 + 1.75 = 8.35**
    3. 유사도의 합을 계산합니다: **0.8 + 0.6 + 0.5 = 1.9**
    4. 최종 예측 평점을 계산합니다: **8.35 / 1.9 = 4.39**

$$\hat{r}_{\text{A, Life of Pi}} = \frac{8.35}{1.9} = 4.39$$

- 유저간 유사도를 이용하면 **(user-based) collaborative filtering**, 아이템간 유사도를 이용하면 **(item-based) collaborative filtering**이라고 함.
- User-based CF의 경우 사용자 간의 유사도를 계산하지만, Item-based는 아이템 간의 유사도를 계산.
- User-based CF는 타겟 사용자와 유사한 사용자들이 좋아한 아이템을 추천하는 반면, Item-based는 사용자가 이미 좋아한 아이템과 유사한 아이템을 추천.
- CF는 단순한 알고리즘이지만 초반에 가볍게 시도하기 좋은 알고리즘이며, 유저나 아이템의 정보가 별도로 주어지지 않아도 추천이 가능하다는 장점.
- 단, 새로운 아이템, 유저에 대해 추천이 어려운 점(Cold-start), 데이터 희소성이 높은 경우 추천퀄리티가 좋지 않고, 사용자수가 증가하면 연산량이 급격히 증가.


## Matrix Factorization
- 대부분의 경우 user-item interaction matrix 는 차원의 수가 불필요하게 많거나 중복이고(노이즈, 매우 희소한(Sparse) 행렬이므로 불필요한 연산량이 많음.
- 모델링에 적합하도록 Dense한 형태로 바꿀 필요가 있는데 차원축소 기법(PCA, SVD)을 이용해 Dense한 형태로 바꾸어 모델링을 진행.
- 차원을 축소한다는 의미는 정보 손실량을 최소로하면서 차원의 수를 줄이는 것을 의미.


- [Matrix Factorization](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 알고리즘은 고차원의 행렬을 저차원의 행렬로 축약하여 예측하는 기법.
- 유저에 대한 잠재적인 요인과 아이템에 대한 잠재적인 요인을 파악하여 평점 예측에 활용.
- 예를 들어, 책 추천 시스템에서 이런 특성들은 '읽기쉬움', '가상현실의', '시간특성' 등일 수 있습니다. 마찬가지로 사용자도 이런 특성에 대한 선호도를 가짐.
- A가 책 B를 얼마나 좋아할지 예측하려면, A의 특성 벡터와 B의 특성 벡터의 내적을 계산합니다. (이 내적 값이 높을수록 A가 B를 좋아할 확률이 높음)
    - **1 x 0.9 +.1 x- .2 = 0.88**
- 실제 평점과 예측 평점의 차이를 최소화하는 방향으로 특성 벡터들을 조정.

In [ ]:
df.head()

In [ ]:
df_mf = df[['user_id', 'isbn', 'rating']].drop_duplicates()
df_mf

In [ ]:
# 30번 이상 rating한 유저만 필터링 > 추천시스템 핵심 타깃으로 정의
# 30번 미만 유저의 경우 다른 approach 이용 > 의사결정 방안
user_list = df_mf.groupby("user_id").size().reset_index()
user_list.columns = ['user_id', 'review_cnt']
user_list = user_list[user_list['review_cnt'] >= 30]

df_mf = df_mf[df_mf['user_id'].isin(user_list.user_id)]

In [ ]:
df_mf.head()

In [ ]:
print(df_mf.shape)
print(df_mf.user_id.nunique())
print(df_mf.isbn.nunique())
print(book_info.isbn.nunique())

In [ ]:
book_info.head()

In [ ]:
# Calculate sparsity
df_sparsity = df_mf.groupby(['user_id', 'isbn']).agg({'rating': 'count'}).reset_index()
user_item_matrix = df_sparsity.pivot(index='user_id', columns='isbn', values='rating')

non_zero_count = user_item_matrix.count().sum()
total_entries = user_item_matrix.size
sparsity = 1 - (non_zero_count / total_entries)

print(f'Interaction size: ', user_item_matrix.count().sum())
print(f'Matrix total size: ', user_item_matrix.size)
print(f'Sparsity of the matrix: {sparsity:.2%}')

In [ ]:
user_item_matrix

In [ ]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV, KFold
from surprise import SVD, SVDpp, NMF, KNNBasic, KNNBaseline, accuracy
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df_mf.sort_values("user_id").head(30)

In [ ]:
#train_data, test_data = train_test_split(data, test_size = 0.3)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_idx, test_idx in split.split(df_mf, df_mf['user_id']):
    strat_train_set = df_mf.iloc[train_idx]  # Use iloc for positional indexing
    strat_test_set = df_mf.iloc[test_idx]

In [ ]:
print(strat_train_set.user_id.nunique())
print(strat_test_set.user_id.nunique())

In [ ]:
strat_train_set.groupby("user_id").size()

In [ ]:
strat_test_set.groupby("user_id").size()

In [ ]:
# Define a Reader object
min_scale = strat_train_set['rating'].min()
max_scale = strat_train_set['rating'].max()

reader = Reader(rating_scale=(min_scale, max_scale))

# Split data into training and test sets
train_data = Dataset.load_from_df(strat_train_set, reader)
trainset = train_data.build_full_trainset()
testset = [tuple(x) for x in strat_test_set.values]

In [ ]:
#testset

In [ ]:
# Define a list of algorithms to test
algorithms = [
    SVD(),
    #KNNBasic(sim_options={"name": "cosine", "user_based": False}),
    KNNBasic(sim_options={"name": "cosine", "user_based": True}),
    KNNBaseline(sim_options={"name": "cosine", "user_based": True}),
    NMF(),
    #SVDpp()
    ]

# Define a function to evaluate a model
def evaluate_model(model, trainset, testset):

    model.fit(trainset)
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)
    return rmse, mae

In [ ]:
# Initialize a list to store results
results = []

# Loop through each algorithm and evaluate
for algorithm in algorithms:
    rmse, mae = evaluate_model(algorithm, trainset, testset)

    print(f"{algorithm.__class__.__name__} Model Evaluation:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print("-" * 30)

    results.append({
        'Algorithm': algorithm.__class__.__name__,
        'RMSE': rmse,
        'MAE': mae
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
# Set the style of the visualization
sns.set(style="whitegrid")

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=True)

# Bar plot for RMSE
sns.barplot(x='Algorithm', y='RMSE', data=results_df, palette='viridis', ax=axes[0])
axes[0].set_title('RMSE for Each Model')
axes[0].tick_params(axis='x', rotation=10)

# Bar plot for MAE
sns.barplot(x='Algorithm', y='MAE', data=results_df, palette='viridis', ax=axes[1])
axes[1].set_title('MAE for Each Model')
axes[1].tick_params(axis='x', rotation=10)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Precision, Recall
min_scale = strat_train_set['rating'].min()
max_scale = strat_train_set['rating'].max()
reader = Reader(rating_scale=(min_scale, max_scale))

train_data = Dataset.load_from_df(strat_train_set[['user_id', 'isbn', 'rating']], reader)
trainset = train_data.build_full_trainset()
testset = [tuple(x) for x in strat_test_set[['user_id', 'isbn', 'rating']].values]

# Fit model and make predictions
model = SVD()
model.fit(train_data.build_full_trainset())
predictions = model.test(testset)

In [ ]:
for x in predictions[:10]:
    print(x)

In [ ]:
# Precsion, Recall check def
top_k = 5
threshold = 7

predictions_df = pd.DataFrame(predictions, columns=['user_id', 'isbn', 'true_rating', 'est_rating', 'details'])
predictions_df = predictions_df[['user_id', 'isbn', 'true_rating', 'est_rating']]

predictions_df['pred_rank'] = predictions_df.groupby('user_id')['est_rating'].rank(method='first', ascending=False)
predictions_df['true_rank'] = predictions_df.groupby('user_id')['true_rating'].rank(method='first', ascending=False)
predictions_df['is_relevant'] = predictions_df['true_rank'] <= top_k

predicted_top_k = predictions_df[predictions_df['pred_rank'] <= top_k]
precision_per_user = predicted_top_k.groupby('user_id')['is_relevant'].sum()

total_relevant_items = predictions_df.query("true_rating >= @threshold").groupby("user_id").size()

precision_at_k = precision_per_user / top_k
recall_at_k = precision_per_user / total_relevant_items

print(precision_at_k.mean(), recall_at_k.mean())

In [ ]:
predictions_df.head()

In [ ]:
#precision_per_user.head()

In [ ]:
#total_relevant_items.head()

In [ ]:
# Define models to test repeatdly
model_classes = {
    'SVD': SVD(),
    'KNNBasic': KNNBasic(sim_options={"name": "cosine", "user_based": True}),
    'KNNBaseline': KNNBaseline(sim_options={"name": "cosine", "user_based": True}),
    'NMF': NMF(),
}

In [ ]:
def get_precision_recall(predictions, top_n, threshold):
    top_k = top_n
    predictions_df = pd.DataFrame(predictions, columns=['user_id', 'isbn', 'true_rating', 'est_rating', 'details'])
    predictions_df = predictions_df[['user_id', 'isbn', 'true_rating', 'est_rating']]

    predictions_df['pred_rank'] = predictions_df.groupby('user_id')['est_rating'].rank(method='first', ascending=False)
    predictions_df['true_rank'] = predictions_df.groupby('user_id')['true_rating'].rank(method='first', ascending=False)
    predictions_df['is_relevant'] = predictions_df['true_rank'] <= top_k
    predictions_df = predictions_df.sort_values(by='user_id')

    predicted_top_k = predictions_df[predictions_df['pred_rank'] <= top_k]
    precision_per_user = predicted_top_k.groupby('user_id')['is_relevant'].sum()
    total_relevant_items = predictions_df.query("true_rating >= @threshold").groupby("user_id").size()

    precision_at_k = precision_per_user / top_k
    recall_at_k = precision_per_user / total_relevant_items

    return precision_at_k.mean(), recall_at_k.mean()

In [ ]:
# Dictionary to store models and their predictions
models_predictions = {}

# Evaluate models and get hit ratios
top_n_values = [1, 2, 3, 4, 5, 6, 7]
precision_results = []
recall_results = []

for model_name, model in model_classes.items():
    print(f"Evaluating {model_name}...")

    # Prepare training data
    train_data = Dataset.load_from_df(strat_train_set[['user_id', 'isbn', 'rating']], reader)
    trainset = train_data.build_full_trainset()
    testset = [tuple(x) for x in strat_test_set[['user_id', 'isbn', 'rating']].values]

    # Fit model and make predictions
    model.fit(train_data.build_full_trainset())
    predictions = model.test(testset)

    # Save the trained model and predictions to the dictionary
    models_predictions[model_name] = {
       'model': model,
        'predictions': predictions
    }

    # Get precision for different Top-N values
    for top_n in top_n_values:
        precision, recall = get_precision_recall(predictions, top_n, 7)
        precision_results.append({'model': model_name, 'topk': top_n, 'precision': precision})
        recall_results.append({'model': model_name, 'topk': top_n,'recall': recall})

In [ ]:
precision_results = pd.DataFrame(precision_results)

In [ ]:
precision_results.head(10)

In [ ]:
precision_results.pivot_table(index='model', columns='topk', values='precision').round(3)

In [ ]:
recall_results = pd.DataFrame(recall_results)

In [ ]:
recall_results.pivot_table(index='model', columns='topk', values='recall').round(3)

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Plot precision for each algorithm
for algorithm in precision_results['model'].unique():
    subset = precision_results[precision_results['model'] == algorithm]
    axes[0].plot(subset['topk'], subset['precision'], marker='o', label=algorithm, alpha=.7)
    axes[0].set_title('Precision of Different Recommendation Algorithms')
    axes[0].set_xlabel('Top-N')
    axes[0].set_ylabel('Precision')
    axes[0].legend(title='Algorithm')
    axes[0].grid(True)


# Plot recall for each algorithm
for algorithm in recall_results['model'].unique():
    subset = recall_results[recall_results['model'] == algorithm]
    axes[1].plot(subset['topk'], subset['recall'], marker='o', label=algorithm, alpha=.7)
    axes[1].set_title('Recall of Different Recommendation Algorithms')
    axes[1].set_xlabel('Top-N')
    axes[1].set_ylabel('Recall')
    axes[1].legend(title='Algorithm')
    axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Number of subplots (equal to number of models)
num_models = len(models_predictions)

# Create subplots (arranged in 2 rows)
fig, axes = plt.subplots(nrows=2, ncols=(num_models + 1) // 2, figsize=(8, 6))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over the models and predictions
for idx, (model_name, model_info) in enumerate(models_predictions.items()):
    # Get the true and predicted ratings
    predictions = model_info['predictions']
    y_true = [true_r[2] for true_r in testset]  # True ratings from testset
    y_pred = [pred.est for pred in predictions]  # Predicted ratings

    # Plot histograms for true and predicted ratings in a subplot
    pd.Series(y_true).hist(alpha=0.7, label='True Ratings', ax=axes[idx])
    pd.Series(y_pred).hist(alpha=0.7, label='Predicted Ratings', ax=axes[idx])

    # Set titles and labels
    axes[idx].set_title(f'{model_name} Histogram')
    axes[idx].legend()
    axes[idx].grid(True)

plt.tight_layout()
plt.show()

- NOTE
    - RMSE, MAE 기준 SVD가 높은 성능(에러 수치이므로 낮은 값일수록 성능 높음)을 보였으나,
    - 랭킹 지표(Recall, Precision)의 경우 KNNBasic이 가장 높은 성능을 보임. (단 NMF 제외하면 차이는 크지 않음)

# 4강 추천알고리즘 성능 고도화
- 일반적으로 베이스라인 모델을 빠르게 구축하고 성능 테스트를 진행/확인한 이후 이를 베이스로하여 성능 고도화를 위한 노력을 진행.
- 데이터를 늘리거나 줄이거나 새로운 데이터를 수집해서 실험해보거나 전처리/후처리 조건 등을 변경하는 작업이 필수.
- 베이스라인 모델의 성능을 개선할 방향성은 크게 아래 2가지.
    - (1) **DeepLearning 계열 DNN, NCF** via TensorFlow
    - (2) **Regression 계열 Model (Linear Regression, Ridge, Random Forest, Gradient Boosting 등)** via Scikit-learn

### Regression 모델
- 머신러닝 모델은 크게 지도학습, 비지도학습 등로 구분되며 지도학습에는 다양한 Regression과 Classification 알고리즘이 있음.
- 추천시스템은 특정 유저의 특정 아이템에 대한 평점을 예측하는 Task이므로 **Regression 모델**을 주로 이용.
- 이번 강의에서는 X,y 간의 선형관계를 가정하는 Linear Regression과 비선형관계를 가정하는 Tree-based 앙상블 기법을 이용.
    - [Linear Regression](https://scikit-learn.org/stable/api/sklearn.linear_model.html)
        - 선형 관계를 가정, Scale 및 아웃라이어에 민감,  데이터가 선형적이면 매우 효과적이고 계산 비용이 적으나
        - 실제 환경에서 선형관계를 가지는 데이터셋을 모델링하는 경우가 흔하지 않음. 예측 성능이 낮은 경향이 있고, 변수의 독립성 등 가정을 잘 지켜야 함
    - [Tree-based Ensemble](https://scikit-learn.org/stable/api/sklearn.ensemble.html)
        - 비선형 관계를 가정, Scale 및 아웃라이어에 큰 영향을 받지 않음. 대부분 실제 환경에서 변수가 간 관계는 비선형 관계를 가짐
        - 계산 비용이 높고, 수식이 상대적으로 복잡하며 다소 블랙박스 형태의 모델이므로 예측 과정 및 이유에 대한 설명력이 높지 않음
    
### Deep Learning 모델
- 신경망은 사람의 뇌를 모방한 구조로, 여러개의 레이어(층)로 구성되어 있으며, 각 레이어는 데이터를 처리하고 더 복잡한 패턴을 찾아냄.
- 데이터를 매우 복잡한 방식으로 처리할 수 있으며, 여러 중간 레이어에서 특징 추출(feature extraction)을 자동으로 수행.
- 특히 이미지, 음성, 자연어 처리 등에서 훨씬 복잡한 패턴을 인식하는 데 매우 뛰어남.


In [ ]:
# DNN, NCF 모델을 이용하여 1차 성능 고도화 시도를 진행합니다.
# Regression 계열 모델을 이용하여 2차 성능 고도화를 진행합니다.
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, BatchNormalization, Dropout

# Preparing user and item embeddings
n_users = df_mf['user_id'].nunique()
n_items = df_mf['isbn'].nunique()

# Creating user and item IDs mapping
user_mapping = {user: idx for idx, user in enumerate(df_mf['user_id'].unique())}
item_mapping = {item: idx for idx, item in enumerate(df_mf['isbn'].unique())}

# Mapping user and item IDs
df_mf['user_id_mapped'] = df_mf['user_id'].map(user_mapping)
df_mf['isbn_mapped'] = df_mf['isbn'].map(item_mapping)

In [ ]:
df_mf.sort_values("user_id").head()

In [ ]:
# Stratified split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_idx, test_idx in split.split(df_mf, df_mf['user_id']):
    strat_train_set = df_mf.iloc[train_idx]  # Use iloc for positional indexing
    strat_test_set = df_mf.iloc[test_idx]

# Splitting the data
X_train = strat_train_set[['user_id_mapped', 'isbn_mapped']]
y_train = strat_train_set['rating']

X_test = strat_test_set[['user_id_mapped', 'isbn_mapped']]
y_test = strat_test_set['rating']

In [ ]:
X_train.sort_index().head()

In [ ]:
strat_train_set.sort_index().head()

In [ ]:
print(X_train.shape)
print(X_test.shape)

## DNN (Deep Neural Network)
- 딥러닝 기반 회귀 모델을 구축하고 학습. (간단한 회귀 문제나 비교적 작은 데이터셋을 다루는 데 적합한 구조로 진행)
- 각 레이어는 ReLU 활성화 함수와 정규화 기법(Batch Normalization, Dropout)을 사용함.
- 입력층에는 128개의 노드가 있으며, 활성화 함수로 ReLU를 사용. ReLU는 음수를 0으로, 양수는 그대로 출력.
- 3개의 중간 레이어 구성: 각 레이어는 Dense 레이어로 구성되며, 차례대로 64, 32, 16개의 노드를 가지고 있음.
- ReLU 활성화 함수와 Batch Normalization, 그리고 20%의 Dropout이 반복적으로 사용.
- 출력층은 1개로 1개의 노드만 가지면 실수를 출력. (평점 예측값)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the model
model = Sequential()

# 입력층
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],))) # fully connected, 레이어 입력층 선언
model.add(BatchNormalization()) # 레이어의 출력을 정규화하여 학습을 안정적으로 만듦
model.add(Dropout(0.2)) # 랜덤하게 20% 가중치 삭제

# 히든 레이어
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# 출력층
model.add(Dense(1)) #출력층으로, 1개의 노드만 가짐. 출력값이 하나(연속형 값)로

# Compile the model
# learning_rate=0.01 학습 속도, 학습이 진행될수록 decay=0.01를 통해 학습률이 점점 감소
optimizer = Adam(learning_rate=0.01, decay=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Define early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001) #검증 데이터의 손실 값(val_loss)이 5번의 에포크 동안 개선되지 않으면 학습 중단. 5번 에포크수 동안, 손실 값이 0.001 미만으로 개선될 경우 변화로 카운팅 제외
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001) #학습률 조절. 검증 손실이 3번의 에포크 동안 개선되지 않으면 학습률을 50% 감소, 최소 학습률은 0.0001

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=256, validation_split=0.2,
          callbacks=[early_stopping, reduce_lr])

In [ ]:
y_pred = model.predict([X_test, X_test])

In [ ]:
y_pred

In [ ]:
# Calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'DNN RMSE: {rmse:.4f}')

mae = mean_absolute_error(y_test, y_pred)  # No square root for MAE
print(f'DNN MAE: {mae:.4f}')

In [ ]:
results_df

In [ ]:
dnn_results = {'Algorithm': 'DNN', 'RMSE': rmse, 'MAE': mae}
dnn_results

In [ ]:
results_df = pd.concat([results_df, pd.DataFrame([dnn_results])], ignore_index=True)
results_df

In [ ]:
# Set the style of the visualization
sns.set(style="whitegrid")

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=False)

# Bar plot for RMSE
sns.barplot(x='Algorithm', y='RMSE', data=results_df, palette='viridis', ax=axes[0])
axes[0].set_title('RMSE for Each Model')
axes[0].tick_params(axis='x', rotation=10)

# Bar plot for MAE
sns.barplot(x='Algorithm', y='MAE', data=results_df, palette='viridis', ax=axes[1])
axes[1].set_title('MAE for Each Model')
axes[1].tick_params(axis='x', rotation=10)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
def get_precision_recall_from_testdata(test_data, top_n, threshold, y_pred):
    top_k = top_n

    predictions_df = test_data.copy()
    predictions_df = predictions_df[['user_id', 'isbn', 'rating']]
    predictions_df['est_rating'] = y_pred

    predictions_df['pred_rank'] = predictions_df.groupby('user_id')['est_rating'].rank(method='first', ascending=False)
    predictions_df['true_rank'] = predictions_df.groupby('user_id')['rating'].rank(method='first', ascending=False)
    predictions_df['is_relevant'] = predictions_df['true_rank'] <= top_k
    predictions_df = predictions_df.sort_values(by='user_id')

    predicted_top_k = predictions_df[predictions_df['pred_rank'] <= top_k]
    precision_per_user = predicted_top_k.groupby('user_id')['is_relevant'].sum()
    total_relevant_items = predictions_df.query("rating >= @threshold").groupby("user_id").size()

    precision_at_k = precision_per_user / top_k
    recall_at_k = precision_per_user / total_relevant_items

    return precision_at_k.mean(), recall_at_k.mean()

In [ ]:
# Dictionary to store models and their predictions
models_predictions = {}

# Evaluate models and get hit ratios
top_n_values = [1, 2, 3, 4, 5, 6, 7]
dnn_precision_results = []
dnn_recall_results = []
threshold = 7

model_name = 'DNN'
# Get precision for different Top-N values
for top_n in top_n_values:
    dnn_precision, dnn_recall = get_precision_recall_from_testdata(strat_test_set, top_n, threshold, y_pred)
    dnn_precision_results.append({'model': model_name, 'topk': top_n, 'precision': dnn_precision})
    dnn_recall_results.append({'model': model_name, 'topk': top_n,'recall': dnn_recall})

In [ ]:
dnn_precision_results

In [ ]:
dnn_recall_results

In [ ]:
precision_results = pd.concat([precision_results, pd.DataFrame(dnn_precision_results)])
recall_results = pd.concat([recall_results, pd.DataFrame(dnn_recall_results)])

In [ ]:
precision_results.pivot_table(index='model', columns='topk', values='precision').round(3)

In [ ]:
recall_results.pivot_table(index='model', columns='topk', values='recall').round(3)

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Plot precision for each algorithm
for algorithm in precision_results['model'].unique():
    subset = precision_results[precision_results['model'] == algorithm]
    axes[0].plot(subset['topk'], subset['precision'], marker='o', label=algorithm, alpha=.7)
    axes[0].set_title('Precision of Different Recommendation Algorithms')
    axes[0].set_xlabel('Top-N')
    axes[0].set_ylabel('Precision')
    axes[0].legend(title='Algorithm')
    axes[0].grid(True)

# Plot recall for each algorithm
for algorithm in recall_results['model'].unique():
    subset = recall_results[recall_results['model'] == algorithm]
    axes[1].plot(subset['topk'], subset['recall'], marker='o', label=algorithm, alpha=.7)
    axes[1].set_title('Recall of Different Recommendation Algorithms')
    axes[1].set_xlabel('Top-N')
    axes[1].set_ylabel('Recall')
    axes[1].legend(title='Algorithm')
    axes[1].grid(True)

plt.tight_layout()
plt.show()

## Neural Collaborative Filtering (NCF)
- 기본적인 모델인 Collaborative Filtering을 신경망 구조로 확장하여, 사용자와 아이템 간의 비선형 관계를 모델링할 수 있도록 설계된 방법.
- NCF는 Matrix Factorization처럼 사용자의 잠재 요인(latent factors)과 아이템의 잠재 요인을 학습.
- 이 과정에서 신경망의 비선형 활성 함수를 사용하여 복잡한 패턴을 학습.
- 전통적인 협업 필터링 기법은 선형 모델에 의존하지만(주로 dot product), NCF는 비선형 활성 함수(예: MLP)를 사용하여 사용자와 아이템 간의 복잡한 관계를 포착.

In [ ]:
# Neural Collaborative Filtering model
n_latent_factors = 64 # 사용자와 아이템의 잠재 요인 수를 정의

user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(n_users, n_latent_factors)(user_input) # 임베딩 진행
user_vec = Flatten()(user_embedding) # 임베딩된 벡터를 1차원 배열로 변환 -> dense layer 입력용

item_embedding = Embedding(n_items, n_latent_factors)(item_input)
item_vec = Flatten()(item_embedding)

# Concatenate user and item embeddings
concat = Concatenate()([user_vec, item_vec]) #사용자 벡터와 아이템 벡터를 연결하여 하나의 벡터로 통합

# Adding fully connected layers with batch normalization and dropout
dense = Dense(256, activation='relu')(concat)
dense = BatchNormalization()(dense)
dense = Dropout(0.2)(dense)

dense = Dense(128, activation='relu')(dense)
dense = BatchNormalization()(dense)
dense = Dropout(0.2)(dense)

output = Dense(1)(dense)

# Compile model
model = Model([user_input, item_input], output)
model.compile(optimizer='rmsprop', loss='mean_squared_error')

# Fit the model
model.fit([X_train['user_id_mapped'], X_train['isbn_mapped']], y_train, epochs=2, batch_size=128, validation_split=0.2)

# Predict and evaluate
y_pred = model.predict([X_test['user_id_mapped'], X_test['isbn_mapped']])

In [ ]:
# Calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'NCF RMSE: {rmse:.4f}')

mae = mean_absolute_error(y_test, y_pred)  # No square root for MAE
print(f'NCF MAE: {mae:.4f}')

In [ ]:
ncf_results = {'Algorithm': 'NCF', 'RMSE': rmse, 'MAE': mae}
ncf_results

In [ ]:
results_df = pd.concat([results_df, pd.DataFrame([ncf_results])], ignore_index=True)
results_df

In [ ]:
# Set the style of the visualization
sns.set(style="whitegrid")

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=False)

# Bar plot for RMSE
sns.barplot(x='Algorithm', y='RMSE', data=results_df, palette='viridis', ax=axes[0])
axes[0].set_title('RMSE for Each Model')
axes[0].tick_params(axis='x', rotation=0)

# Bar plot for MAE
sns.barplot(x='Algorithm', y='MAE', data=results_df, palette='viridis', ax=axes[1])
axes[1].set_title('MAE for Each Model')
axes[1].tick_params(axis='x', rotation=0)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Dictionary to store models and their predictions
models_predictions = {}

# Evaluate models and get hit ratios
top_n_values = [1, 2, 3, 4, 5, 6, 7]
ncf_precision_results = []
ncf_recall_results = []
threshold = 7

model_name = 'NCF'
# Get precision for different Top-N values
for top_n in top_n_values:
    ncf_precision, ncf_recall = get_precision_recall_from_testdata(strat_test_set, top_n, threshold, y_pred)
    ncf_precision_results.append({'model': model_name, 'topk': top_n, 'precision': ncf_precision})
    ncf_recall_results.append({'model': model_name, 'topk': top_n,'recall': ncf_recall})

In [ ]:
precision_results = pd.concat([precision_results, pd.DataFrame(ncf_precision_results)])
recall_results = pd.concat([recall_results, pd.DataFrame(ncf_recall_results)])

In [ ]:
precision_results.pivot_table(index='model', columns='topk', values='precision').round(3)

In [ ]:
recall_results.pivot_table(index='model', columns='topk', values='recall').round(3)

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Plot precision for each algorithm
for algorithm in precision_results['model'].unique():
    subset = precision_results[precision_results['model'] == algorithm]
    axes[0].plot(subset['topk'], subset['precision'], marker='o', label=algorithm, alpha=.7)
    axes[0].set_title('Precision of Different Recommendation Algorithms')
    axes[0].set_xlabel('Top-N')
    axes[0].set_ylabel('Precision')
    axes[0].legend(title='Algorithm')
    axes[0].grid(True)

# Plot recall for each algorithm
for algorithm in recall_results['model'].unique():
    subset = recall_results[recall_results['model'] == algorithm]
    axes[1].plot(subset['topk'], subset['recall'], marker='o', label=algorithm, alpha=.7)
    axes[1].set_title('Recall of Different Recommendation Algorithms')
    axes[1].set_xlabel('Top-N')
    axes[1].set_ylabel('Recall')
    axes[1].legend(title='Algorithm')
    axes[1].grid(True)

plt.tight_layout()
plt.show()

## Machine Learning(Regression) Model

- 딥러닝 모델의 경우 비정형 데이터(이미지, 오디오, 텍스트 등)에서 스스로 피처를 추출하고 복잡한 관계를 학습하는데 특화된 반면
- 간단한 데이터나 Tabular 데이터셋에는 잘 맞지 않는 경향이 있음.
- 이 경우 Linear Regression 과 같은 전통적인 구조가 간단하여 속도가 빠르고 설명력이 높고 리소스 및 운영 코스트 효율이 높은 머신러닝 모델을 주로 이용.
- 수많은 Regression 알고리즘 중에서 몇개를 선정하여 아래와 같이 딕셔너리를 선언하여 각 성능을 체크.




In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Regression 알고리즘 후보 (이중 높은 성능을 보이는 모델을 실험을 통해 빠르게 찾는 과정이 중요)
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'ExtraTreesRegressor': ExtraTreesRegressor(),
}

models

In [ ]:
df.head()

In [ ]:
# Preprocessing
df_mf = df[['user_id', 'isbn', 'rating']].drop_duplicates()

user_list = df_mf.groupby("user_id").size().reset_index()
user_list.columns = ['user_id', 'review_cnt']
user_list = user_list[user_list['review_cnt'] >= 30]

df_mf = df_mf[df_mf['user_id'].isin(user_list.user_id)]

In [ ]:
df_mf.head()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_idx, test_idx in split.split(df_mf, df_mf['user_id']):
    strat_train_set = df_mf.iloc[train_idx]
    strat_test_set = df_mf.iloc[test_idx]

In [ ]:
strat_train_set.head()

In [ ]:
print(strat_train_set.user_id.nunique())
print(strat_test_set.user_id.nunique())

In [ ]:
book_info.head()

In [ ]:
user_info = df[['user_id', 'age']].drop_duplicates()
user_info.head()

In [ ]:
def encode_categorical_columns(df):
    categorical_columns = ['isbn', 'book_author', 'category']
    for column in categorical_columns:
        df[f'{column}_encoded'] = df[column].astype('category').cat.codes
    return df

def prepare_data(df, book_info, user_info):
    merged_df = df.merge(book_info, on=['isbn'])\
                  .merge(user_info[['user_id', 'age']].drop_duplicates(), on='user_id')
    encoded_df = encode_categorical_columns(merged_df)
    return encoded_df

strat_train_set = prepare_data(strat_train_set, book_info, user_info)
strat_test_set = prepare_data(strat_test_set, book_info, user_info)

In [ ]:
strat_train_set.head(2)

In [ ]:
def split_data(df):
    features = ['age', 'isbn_encoded', 'book_author_encoded', 'year_of_publication',
                'rating_mean', 'rating_count', 'category_encoded']

    X = df[features]
    y = df['rating']
    return X, y

X_train, y_train = split_data(strat_train_set)
X_test, y_test = split_data(strat_test_set)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_test.head()

In [ ]:
models

In [ ]:
results = {}
# Iterate over the models and calculate the MAE and RMSE values
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    #print(name, model)
    results[name] = {'MAE': mae, 'RMSE': rmse}

In [ ]:
pd.DataFrame(results)

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(results.keys(), [value['RMSE'] for value in results.values()])
plt.ylabel('RMSE')
plt.xticks(rotation=15)
plt.title('RMSE Values for Each Model')

plt.subplot(1, 2, 2)
plt.bar(results.keys(), [value['MAE'] for value in results.values()])
plt.xticks(rotation=15)
plt.ylabel('MAE')
plt.title('MAE Values for Each Model')

plt.tight_layout()
plt.show()

In [ ]:
# Set the style of the visualization
sns.set(style="whitegrid")

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=False)

# Bar plot for RMSE
sns.barplot(x='Algorithm', y='RMSE', data=results_df, palette='viridis', ax=axes[0])
axes[0].set_title('RMSE for Each Model')
axes[0].tick_params(axis='x', rotation=0)

# Bar plot for MAE
sns.barplot(x='Algorithm', y='MAE', data=results_df, palette='viridis', ax=axes[1])
axes[1].set_title('MAE for Each Model')
axes[1].tick_params(axis='x', rotation=0)

# Adjust layout
plt.tight_layout()
plt.show()

- Note
    - Feature 기반의 모델이 성능이 좋으나 항상 Feature 정보가 있는 것은 아니며,
    - Feature Quailty에 의해 성능이 크게 영향을 받는다는 단점이 존재. (예, 장애 및 수집오류로 인한 이슈)
    - 상황에 맞춰 두 방향성을 모두 고려하여 최적화하여 이용하는 것이 필요. (예, ML 모델의 보안으로 cf, mf 이용 등)

In [ ]:
models

In [ ]:
# Dictionary to store models and their predictions
models_predictions = {}

# Evaluate models and get hit ratios
top_n_values = [1, 2, 3, 4, 5, 6, 7]
ml_precision_results = []
ml_recall_results = []
threshold = 7

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    models_predictions[model_name] = {
       'model': model,
        'predictions': y_pred
    }

    # Get precision for different Top-N values
    for top_n in top_n_values:
        ml_precision, ml_recall = get_precision_recall_from_testdata(strat_test_set, top_n, threshold, y_pred)
        ml_precision_results.append({'model': model_name, 'topk': top_n, 'precision': ml_precision})
        ml_recall_results.append({'model': model_name, 'topk': top_n,'recall': ml_recall})

In [ ]:
ml_precision_results = pd.DataFrame(ml_precision_results)
ml_precision_results.head()

In [ ]:
ml_recall_results = pd.DataFrame(ml_recall_results)
ml_recall_results.tail()

In [ ]:
ml_precision_results.pivot_table(index='model', columns='topk', values='precision')

In [ ]:
ml_recall_results.pivot_table(index='model', columns='topk', values='recall')

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Plot precision for each algorithm
for algorithm in ml_precision_results['model'].unique():
    subset = ml_precision_results[ml_precision_results['model'] == algorithm]
    axes[0].plot(subset['topk'], subset['precision'], marker='o', label=algorithm, alpha=.7)
    axes[0].set_title('Precision of Different Recommendation Algorithms')
    axes[0].set_xlabel('Top-N')
    axes[0].set_ylabel('Precision')
    axes[0].legend(title='Algorithm')
    axes[0].grid(True)

# Plot recall for each algorithm
for algorithm in ml_recall_results['model'].unique():
    subset = ml_recall_results[ml_recall_results['model'] == algorithm]
    axes[1].plot(subset['topk'], subset['recall'], marker='o', label=algorithm, alpha=.7)
    axes[1].set_title('Recall of Different Recommendation Algorithms')
    axes[1].set_xlabel('Top-N')
    axes[1].set_ylabel('Recall')
    axes[1].legend(title='Algorithm')
    axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Plot precision for each algorithm
for algorithm in precision_results['model'].unique():
    subset = precision_results[precision_results['model'] == algorithm]
    axes[0].plot(subset['topk'], subset['precision'], marker='o', label=algorithm, alpha=.7)
    axes[0].set_title('Precision of Different Recommendation Algorithms')
    axes[0].set_xlabel('Top-N')
    axes[0].set_ylabel('Precision')
    axes[0].legend(title='Algorithm')
    axes[0].grid(True)

# Plot recall for each algorithm
for algorithm in recall_results['model'].unique():
    subset = recall_results[recall_results['model'] == algorithm]
    axes[1].plot(subset['topk'], subset['recall'], marker='o', label=algorithm, alpha=.7)
    axes[1].set_title('Recall of Different Recommendation Algorithms')
    axes[1].set_xlabel('Top-N')
    axes[1].set_ylabel('Recall')
    axes[1].legend(title='Algorithm')
    axes[1].grid(True)

plt.tight_layout()
plt.show()

# 5강 Content-Based Filtering & Popularity
- Content-Based Filtering 은 이전 성능을 비교했던 추천 알고리즘과 다소 목적이 다름.
- 새로운 유저, 아이템에 대해 추천할 내용이 없는 경우 Cold Start 문제라고 하며 이는 추천시스템에서 반드시 다루어야할 과제.
- Cold Start 문제를 사용자/아이템 프로파일링, 군집화를 이용해 해결할 수 있으나, 이를 위해 정교한 추가 데이터 분석이 필요.
- Content-Based Filtering과 Popularity-based 추천을 통해 해결하는 과정을 진행.

## Content-based filtering
- 유저 과거에 혹은 최근에 선호했던 아이템의 특성을 기반으로 비슷한 특성의 새로운 아이템을 추천하는 방법.
  - 1) 유저가 선호하는 아이템을 정의하고,
  - 2) 해당 아이템과 유사한 다른 아이템을 찾아서 추천합니다(유사도가 높은 순서대로)
- 아이템 별로 임베딩을 진행하고 모든 아이템 x 아이템 쌍별로 코사인 유사도를 이용해 유사도 기준으로 유사 아이템을 찾아낼수 있음.
- 개인화된 추천 아이템을 제공하나, 유사한 아이템이 중복적으로 추천되므로 새로운 아이템(Novelty, Serendipity)이 부족할수 있음.
- 따라서 단독 모델로 쓰이기 보다, 보완용 혹은 하이브리드용으로 주로 쓰임.

## Popularity-based 추천
- 비개인화된 추천이며, 가장 많이 팔린 아이템, 평점이 높은 아이템 순으로 정렬하여 누구에게나 동일하게 추천하는 단순한 방식.
- 개인화가 반영되지 않아 성능이나 사용자 피드백이 좋지 않은 경향이 있어, 사용하지 않는 추세.

In [ ]:
# Content-based filtering
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [ ]:
#!pip install nltk

In [ ]:
df.head(2)

In [ ]:
book_info.head(2)

In [ ]:
# book selection
book_rec = book_info[(book_info['rating_count'] >= 3) & (book_info['rating_mean'] >= 4)]

# summary added
book_rec = book_rec.merge(df[['isbn', 'summary']].drop_duplicates(), on='isbn', how='left').fillna("")

In [ ]:
book_rec.head()

In [ ]:
# Function to clean and extract meaningful keywords
def clean_text(summary):
    # Remove punctuation, numbers, and special characters using regex
    cleaned_summary = re.sub(r'[^a-zA-Z\s]', '', summary)

    # Tokenize the summary by splitting on whitespace
    words = cleaned_summary.lower().split()

    # Filter out stopwords and short words (less than 3 characters)
    meaningful_words = [word for word in words if word not in stop_words and len(word) > 2]

    return ' '.join(meaningful_words)  # Return as a single string

In [ ]:
# Apply the cleaning function to the 'summary' column
book_rec['cleaned_summary'] = book_rec['summary'].apply(clean_text)
book_rec['keywords'] = book_rec['book_title']+' '+book_rec['book_author']+' '+book_rec['cleaned_summary']+' '+book_rec['category']

book_rec = book_rec[['isbn', 'book_title', 'keywords']]
book_rec['keywords'] = book_rec['keywords'].apply(clean_text)

In [ ]:
book_rec.head(2)

In [ ]:
book_rec.shape

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_features=1000, min_df=10)
tfidf_matrix = tfidf.fit_transform(book_rec['keywords'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
tfidf_matrix

In [ ]:
cosine_sim

In [ ]:
book_rec[book_rec['isbn'] == my_isbn]

In [ ]:
# 0385504209, 059035342X, 0345370775, 0345337662, 0345339681
my_isbn = '0345339681' # 추천기준 아이템
top_n = 10

idx = book_rec[book_rec['isbn'] == my_isbn].index[0]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:top_n+1]
book_indices = [i[0] for i in sim_scores]
book_rec.iloc[book_indices]

In [ ]:
book_rec[book_rec['isbn'] == my_isbn]

In [ ]:
#3426600218, 0385504209, 059035342X, 0345370775, 0345337662, 0345339681
my_isbn = '3426600218'
top_n = 10

def get_recom_items(my_isbn, top_n, cosine_sim, book_rec, std_book_include=True):

    idx = book_rec[book_rec['isbn'] == my_isbn].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    if std_book_include:
        sim_scores = sim_scores[:top_n+1]
    else:
        sim_scores = sim_scores[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]

    return book_rec.iloc[book_indices]

In [ ]:
recom_result = get_recom_items(my_isbn, top_n, cosine_sim, book_rec, std_book_include=False)
recom_result

- Note
  - 보통 유저별 최근 구매 아이템중에 빈도가 많거나 평가 좋은 아이템을 기준으로 이용
  - 없는 경우, 자주 클릭한 아이템을 이용하거나 세그먼트별 대표아이템을 대안으로 이용
  - 그래도 없으면 회원가입시 간단한 설문조사로 파악(e.g. Netflix)

In [ ]:
# Popularity Recommendation, but NON-personalized
category_list = book_info.groupby(['category']).size()
category_list.sort_values(ascending=False)[:10]

In [ ]:
#my_cate = 'Business & Economics'
my_cate = 'Business & Economics'
book_info.query("category == @my_cate").sort_values("rating_count", ascending=False)[:10]

In [ ]:
book_rec.head()

In [ ]:
df_mf['user_id']

In [ ]:
# Content-based & Populiarty Hybrid
user_id = 219008
top_n = 10

def get_recommended_items_content(user_id, top_n):

    # the previous book that the user liked the most
    most_liked_isbn = df_mf[df_mf['user_id'] == user_id].sort_values("rating", ascending=False)['isbn']
    if most_liked_isbn.empty:
        return pd.DataFrame()
    most_liked_isbn = most_liked_isbn.values[0] # get the isbm

    # popularity-based if none
    if most_liked_isbn not in book_rec['isbn'].values:
        predictions = book_info.sort_values("rating_count", ascending=False)[:top_n]
        predictions['user_id'] = user_id
        predictions = predictions[['user_id', 'isbn', 'book_title', 'book_author']]

    # using content-based
    else:
        idx = book_rec[book_rec['isbn'] == most_liked_isbn].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
        book_indices = [i[0] for i in sim_scores]
        recommend_books = book_rec.iloc[book_indices]['isbn']
        predictions = book_info[book_info['isbn'].isin(recommend_books)]
        predictions['user_id'] = user_id
        predictions = predictions[['user_id', 'isbn', 'book_title', 'book_author']]
    return predictions

get_recommended_items_content(user_id, top_n)

# 6강 최종 추천모델 클래스 구성
- 지금까지 3실험한 알고리즘중 일부를 선정해 하나의 추천 클래스를 구성.
- 클래스 하나에 **Matrix Factorization(SVD), Machine Learning(GBR), Content+Poupularity** 추천을 생성하는 코드를 작성.

## SVD Recommender

In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from sklearn.ensemble import GradientBoostingRegressor

class SVDRecommender:
    def __init__(self, df_mf):
        self.df_mf = df_mf
        self.svd_model = self.fit_svd_model()

    def fit_svd_model(self):
        # Preprocessing
        df_mf = self.df_mf[['user_id', 'isbn', 'rating']].drop_duplicates()
        user_list = df_mf.groupby("user_id").size().reset_index(name='review_cnt')
        user_list = user_list[user_list['review_cnt'] >= 30]

        df_mf = df_mf[df_mf['user_id'].isin(user_list['user_id'])]

        # Ensure reader is defined
        reader = Reader(rating_scale=(1, 5))  # Adjust scale as needed

        # Train
        train_data = Dataset.load_from_df(df_mf[['user_id', 'isbn', 'rating']], reader)
        trainset = train_data.build_full_trainset()
        svd = SVD()
        svd.fit(trainset)

        return svd

## GBR Recommender

In [ ]:
class GBRRecommender:
    def __init__(self, ml_train_set, book_info, user_info):
        self.ml_train_set = ml_train_set
        self.book_info = book_info
        self.user_info = user_info
        self.gbr_model = self.fit_gbr_model()

    def fit_gbr_model(self):
        # Prepare data for GBR fitting
        df_mf = self.preprocessing(self.ml_train_set)

        # Train
        X_train, y_train = self.split_data(df_mf)
        gbr_model = GradientBoostingRegressor(random_state=42)
        gbr_model.fit(X_train, y_train)

        return gbr_model

    def preprocessing(self, df):
        df_mf = df[['user_id', 'isbn', 'rating']].drop_duplicates()
        user_list = df_mf.groupby("user_id").size().reset_index(name='review_cnt')
        user_list = user_list[user_list['review_cnt'] >= 30]

        df_mf = df_mf[df_mf['user_id'].isin(user_list['user_id'])]

        merged_df = df_mf.merge(self.book_info, on='isbn')\
                          .merge(self.user_info[['user_id', 'age']].drop_duplicates(), on='user_id')
        return self.encode_categorical_columns(merged_df)

    def encode_categorical_columns(self, df):
        """Encode categorical columns in the DataFrame."""
        categorical_columns = ['isbn', 'book_author', 'category']
        for column in categorical_columns:
            df[f'{column}_encoded'] = df[column].astype('category').cat.codes
        return df

    def split_data(self, df):

        features = ['age', 'isbn_encoded', 'book_author_encoded', 'year_of_publication',
                    'rating_mean', 'rating_count', 'category_encoded']

        # Check if all features exist in the DataFrame
        missing_features = [feature for feature in features if feature not in df.columns]
        if missing_features:
            print(f"Missing features: {missing_features}")
            raise KeyError(f"None of {missing_features} are in the input DataFrame")

        X = df[features]
        y = df['rating']
        return X, y

## Final Book Recommender with SVD, GBR, Content&Popularity
- SVD Recommender와 GBR Recommender 클래스를 상속받아 Book Recommender 에서는 3개의 알고리즘을 이용해 추천셋 생성

In [ ]:
class BookRecommender(SVDRecommender, GBRRecommender):

    def __init__(self, df_mf, book_info, book_rec, ml_train_set, cosine_sim):
        SVDRecommender.__init__(self, df_mf)
        GBRRecommender.__init__(self, ml_train_set, book_info, ml_train_set[['user_id', 'age']].drop_duplicates())

        self.book_info = book_info
        self.book_rec = book_rec
        self.cosine_sim = cosine_sim

    def get_recommended_items_mf(self, user_id, top_n):
        """Matrix Factorization based recommendations"""
        user_reviewed_iid = self.df_mf[self.df_mf['user_id'] == user_id]['isbn']
        iid_to_est = self.df_mf[~self.df_mf['isbn'].isin(user_reviewed_iid)]['isbn'].drop_duplicates()
        testset = [(user_id, iid, None) for iid in iid_to_est]

        predictions = self.svd_model.test(testset)
        predictions = pd.DataFrame(predictions)[['uid', 'iid', 'est']]
        predictions.columns = ['user_id', 'isbn', 'predicted_rating']
        predictions['predicted_rank'] = predictions['predicted_rating'].rank(method='first', ascending=False)
        predictions = predictions.sort_values("predicted_rank")[:top_n]
        predictions = predictions.merge(self.book_info, on='isbn')

        return predictions[['user_id', 'isbn', 'book_title', 'book_author']]

    def get_recommended_items_ml(self, user_id, top_n):
        """Machine Learning (GBR) based recommendations"""
        user_reviewed_iid = self.df_mf[self.df_mf['user_id'] == user_id]['isbn']
        iid_to_est = self.df_mf[~self.df_mf['isbn'].isin(user_reviewed_iid)]['isbn'].drop_duplicates()
        ml_test_set = self.ml_train_set[self.ml_train_set['isbn'].isin(iid_to_est)]
        user_age = self.ml_train_set[self.ml_train_set['user_id'] == user_id]['age'].unique()

        features = ['age', 'isbn_encoded', 'book_author_encoded', 'year_of_publication',
                    'rating_mean', 'rating_count', 'category_encoded']

        X = ml_test_set[features].query("age == @user_age")
        y_pred = self.gbr_model.predict(X)

        gbr_rec_result = X.copy()
        gbr_rec_result['y_pred'] = y_pred
        gbr_rec_result = gbr_rec_result.sort_values("y_pred", ascending=False)[:top_n]
        gbr_rec_result = gbr_rec_result.merge(self.ml_train_set[['isbn_encoded', 'isbn', 'book_title', 'book_author']].drop_duplicates(), on='isbn_encoded')
        gbr_rec_result = gbr_rec_result[['isbn', 'book_title', 'book_author']]
        gbr_rec_result['user_id'] = user_id
        gbr_rec_result = gbr_rec_result.reindex(columns=['user_id', 'isbn', 'book_title', 'book_author'])

        return gbr_rec_result

    def get_recommended_items_content(self, user_id, top_n):
        """ Content-based recommendations """
        most_liked_isbn = self.df_mf[self.df_mf['user_id'] == user_id].sort_values("rating", ascending=False)['isbn']

        if most_liked_isbn.empty:
            return pd.DataFrame()  # Return empty DataFrame if user has no ratings

        most_liked_isbn = most_liked_isbn.values[0]

        if most_liked_isbn not in self.book_rec['isbn'].values:
            predictions = self.book_info.sort_values("rating_count", ascending=False)[:top_n]
            predictions['user_id'] = user_id
            return predictions[['user_id', 'isbn', 'book_title', 'book_author']]
        else:
            idx = self.book_rec[self.book_rec['isbn'] == most_liked_isbn].index[0]
            sim_scores = list(enumerate(self.cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
            book_indices = [i[0] for i in sim_scores]
            recommend_books = self.book_rec.iloc[book_indices]['isbn']
            predictions = self.book_info[self.book_info['isbn'].isin(recommend_books)]
            predictions['user_id'] = user_id
            return predictions[['user_id', 'isbn', 'book_title', 'book_author']]


book_recommender = BookRecommender(df_mf, book_info, book_rec, strat_train_set, cosine_sim)

In [ ]:
# randome user_ids
unique_user_ids = df_mf['user_id'].unique()
np.random.shuffle(unique_user_ids)
print(unique_user_ids[:20])

In [ ]:
user_id = 28255
top_n = 10

mf_rec_result = book_recommender.get_recommended_items_mf(user_id, top_n)
mf_rec_result

In [ ]:
ml_rec_result = book_recommender.get_recommended_items_ml(user_id, top_n)
ml_rec_result

In [ ]:
content_rec_result = book_recommender.get_recommended_items_content(user_id, top_n)
content_rec_result